In [ ]:
import subprocess

!pip install faiss-cpu
!sudo apt-get install python3.10
!sudo update-alternatives --install "/usr/bin/python3" python3 "/usr/bin/python3.10" 1
!sudo update-alternatives --install "/usr/bin/python3" python3 "/usr/bin/python3.11" 0
!sudo apt-get install python3-pip

packages = ['build-essential', 'python3-dev', 'ffmpeg', 'aria2']
pip_packages = ['setuptools', 'wheel', 'fairseq', 'ffmpeg', 'ffmpeg-python', 'praat-parselmouth', 'pyworld', 'numpy==1.23.5', 'numba==0.56.4', 'librosa==0.9.2', 'matplotlib==3.7.0', 'tensorboard']

print("Updating and installing system packages...")
for package in packages:
  print(f"Installing {package}...")
  subprocess.check_call(['apt-get', 'install', '-qq', '-y', package])

print("Updating and installing pip packages...")

for package in pip_packages:
  print(f"Installing {package}...")
  subprocess.check_call(['pip', 'install', '--upgrade', package])

!pip uninstall torch torchvision torchaudio -y
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --upgrade --index-url https://download.pytorch.org/whl/cu121

print('Packages up to date.')
firsttry = True

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 31.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3.10 is already the newest version (3.10.12-1~22.04.9).
python3.10 set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
update-alternatives: using /usr/bin/python3.10 to provide /usr/bin/python3 (python3) in auto mode
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3-setuptools python3-wheel
Suggested packages:
  python-setuptools-doc
The following NEW packages will be installed:
  python3-pip python3-setuptools python3-wheel
0 upgraded, 3 newly installed, 0 to remove and 34 not upgraded.
Need to get 1,677 kB of archives.
After this operation, 8,968 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 python3-

In [ ]:
import os

os.chdir('/content/')

if(os.path.exists("/content/Mangio-RVC-Fork")):
  print("RVC already installed, skipping.")
else:
  !git clone https://github.com/Mangio621/Mangio-RVC-Fork.git
  !git clone https://github.com/maxrmorrison/torchcrepe.git
  !mv torchcrepe/torchcrepe Mangio-RVC-Fork/
  !rm -rf torchcrepe  # Delete the torchcrepe repository folder

os.chdir('/content/Mangio-RVC-Fork')
now_dir = "/content/Mangio-RVC-Fork"
os.makedirs(os.path.join(now_dir, "logs"), exist_ok=True)
os.makedirs(os.path.join(now_dir, "weights"), exist_ok=True)

Cloning into 'Mangio-RVC-Fork'...
remote: Enumerating objects: 2969, done.
remote: Total 2969 (delta 0), reused 0 (delta 0), pack-reused 2969 (from 1)
Receiving objects: 100% (2969/2969), 13.75 MiB | 14.68 MiB/s, done.
Resolving deltas: 100% (1871/1871), done.
Cloning into 'torchcrepe'...
remote: Enumerating objects: 457, done.
remote: Counting objects: 100% (452/452), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 457 (delta 264), reused 412 (delta 249), pack-reused 5 (from 1)
Receiving objects: 100% (457/457), 72.21 MiB | 5.82 MiB/s, done.
Resolving deltas: 100% (264/264), done.


In [ ]:
import torch

ngpu = torch.cuda.device_count()
gpu_infos = []
mem = []
if_gpu_ok = False

if torch.cuda.is_available() or ngpu != 0:
  for i in range(ngpu):
    gpu_name = torch.cuda.get_device_name(i)
    if any(
        value in gpu_name.upper()
        for value in ["10", "16", "20", "30", "40", "A2", "A3", "A4", "P4", "A50", "500", "A60", "70", "80", "90", "M4", "T4", "TITAN"]
    ):
      if_gpu_ok = True
      print("Compatible GPU detected: %s" % gpu_name)
      gpu_infos.append("%s\t%s" % (i, gpu_name))
      mem.append(int(torch.cuda.get_device_properties(i).total_memory / 1024 / 1024 / 1024 + 0.4))

if if_gpu_ok and len(gpu_infos) > 0:
  gpu_info = "\n".join(gpu_infos)

else:
  raise Exception("No GPU detected; training cannot continue. Please change your runtime type to a GPU.")
gpus = "-".join(i[0] for i in gpu_infos)

Compatible GPU detected: Tesla T4


In [ ]:
from google.colab import drive
if not os.path.exists('/content/drive'):
  drive.mount('/content/drive')
else:
  print('Drive is already mounted. Proceed.')

os.makedirs('/content/drive/MyDrive/rvcDisconnected', exist_ok=True)

Mounted at /content/drive


In [ ]:
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/hubert_base.pt -d /content/Mangio-RVC-Fork -o hubert_base.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/rmvpe.pt -d /content/Mangio-RVC-Fork -o rmvpe.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/40k.json -d /content/Mangio-RVC-Fork/configs -o 40k.json



Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
f782dd|OK  |   258MiB/s|/content/Mangio-RVC-Fork/hubert_base.pt

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
e942d4|OK  |   263MiB/s|/content/Mangio-RVC-Fork/rmvpe.pt

Status Legend:
(OK):download completed.
--2025-05-08 07:59:49--  https://dl.fbaipublicfiles.com/hubert/hubert_large_ll60k_finetune_ls960.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.227.74.12, 13.227.74.9, 13.227.74.45, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.227.74.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3790461440 (3.5G) [application/zip]
Saving to: ‘hubert_large_ll60k_finetune_ls960.pt’

hubert_large_ll60k_ 100%[===================>]   3.53G  77.0MB/s    in 45s     

2025-05-0

In [ ]:
#@title Setup CSVDB
#...Alright, you made your point.
import csv

if not os.path.isdir("csvdb/"):
  os.makedirs("csvdb")
  frmnt, stp = open("csvdb/formanting.csv", "w", newline=""), open("csvdb/stop.csv", "w", newline="")
  csv_writer = csv.writer(frmnt, delimiter=",")
  csv_writer.writerow([False, 1.0, 1.0])
  csv_writer = csv.writer(stp, delimiter=",")
  csv_writer.writerow([False])
  frmnt.close()
  stp.close()

global DoFormant, Quefrency, Timbre
DoFormant, Quefrency, Timbre = False, 1.0, 1.0

In [ ]:
now_dir = "/content/Mangio-RVC-Fork"
experiment_name = "viveknewer"
path_to_training_folder = "/content/dataset/"
model_architecture = "v2"
target_sample_rate = "40k"
speaker_id = 0
pitch_extraction_algorithm = "rmvpe"
crepe_hop_length = 64
pitch_guidance = True

cpu_threads = !nproc
cpu_threads = int(cpu_threads[0])

exp_dir = f"{now_dir}/logs/{experiment_name}"

assert crepe_hop_length!=None,
assert crepe_hop_length>0,
assert crepe_hop_length<=512,

if(experiment_name == "experiment_name"):
  print("Warning: Your experiment name should be changed to the name of your dataset.")

In [ ]:
# Set required values
target_sample_rate = "40k"  # Change to "32k" or "48k" if needed and available
model_architecture = "v2"

# Set TITAN model URLs
g = f"https://huggingface.co/blaise-tk/TITAN/resolve/main/models/medium/{target_sample_rate}/pretrained/G-f0{target_sample_rate}-TITAN-Medium.pth"
d = f"https://huggingface.co/blaise-tk/TITAN/resolve/main/models/medium/{target_sample_rate}/pretrained/D-f0{target_sample_rate}-TITAN-Medium.pth"

# Set download path based on architecture
pretrained_base = "pretrained_v2/"

# Create path if it doesn't exist (only necessary outside Colab)
import os
os.makedirs(f"/content/Mangio-RVC-Fork/{pretrained_base}", exist_ok=True)

# Download using aria2c
print("Downloading TITAN pretrained model...")
!aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M {g} -d /content/Mangio-RVC-Fork/{pretrained_base} -o f0G{target_sample_rate}_TITAN.pth
!aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M {d} -d /content/Mangio-RVC-Fork/{pretrained_base} -o f0D{target_sample_rate}_TITAN.pth
print("TITAN pretrained model downloaded successfully.")


In [ ]:
dataset = "vivek2.0.zip"

import os
import shutil

directories=[]

def sanitize_directory(directory):
  for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    if os.path.isfile(file_path):
      if filename == ".DS_Store" or filename.startswith("._") or not filename.endswith(('.wav', '.flac', '.mp3', '.ogg', '.m4a')):
        os.remove(file_path)
    elif os.path.isdir(file_path):
      if(filename == "__MACOSX"):
        shutil.rmtree(file_path)
        continue
      directories.append(file_path)
      sanitize_directory(file_path)

dataset_path = '/content/drive/MyDrive/rvcDisconnected/' + dataset
final_directory = '/content/dataset'
temp_directory = '/content/temp_dataset'

if os.path.exists(final_directory):
  print("Dataset folder already found. Wiping...")
  shutil.rmtree(final_directory)
if os.path.exists(temp_directory):
  print("Temporary folder already found. Wiping...")
  shutil.rmtree(temp_directory)

if not os.path.exists(dataset_path):
  raise Exception(f'I can\'t find {dataset} in {os.path.dirname(dataset_path)}.')

os.makedirs(final_directory, exist_ok=True)
os.makedirs(temp_directory, exist_ok=True)
#Oops.
!unzip -d "{temp_directory}" -B "{dataset_path}"
print("Sanitizing...")
sanitize_directory(temp_directory)

if(len(directories) == 0):
  print("Dataset Type: Audio Files (Single Speaker)")
  expDir=os.path.join(final_directory, experiment_name)
  os.makedirs(expDir, exist_ok=True)
  for r, _, f in os.walk(temp_directory):
    for name in f:
      !cp "{temp_directory}/{name}" "{expDir}"
elif(len(directories) == 1):
  print("Dataset Type: Single Speaker")
  fi = os.path.join(temp_directory, experiment_name)
  os.rename(directories[0], fi)
  shutil.move(fi, final_directory)

else:
  print("Dataset Type: Multispeaker")
  for fi in directories:
    shutil.move(fi, final_directory)

shutil.rmtree(temp_directory)

print("Dataset imported.")


Archive:  /content/drive/MyDrive/rvcDisconnected/vivek2.0.zip
  inflating: /content/temp_dataset/16.wav  
  inflating: /content/temp_dataset/1.wav  
  inflating: /content/temp_dataset/21.wav  
  inflating: /content/temp_dataset/25.wav  
  inflating: /content/temp_dataset/24.wav  
  inflating: /content/temp_dataset/15.wav  
  inflating: /content/temp_dataset/20.wav  
  inflating: /content/temp_dataset/27.wav  
  inflating: /content/temp_dataset/18.wav  
  inflating: /content/temp_dataset/26.wav  
  inflating: /content/temp_dataset/8.wav  
  inflating: /content/temp_dataset/9.wav  
  inflating: /content/temp_dataset/11.wav  
  inflating: /content/temp_dataset/6.wav  
  inflating: /content/temp_dataset/22.wav  
  inflating: /content/temp_dataset/10.wav  
  inflating: /content/temp_dataset/13.wav  
  inflating: /content/temp_dataset/12.wav  
  inflating: /content/temp_dataset/17.wav  
  inflating: /content/temp_dataset/2.wav  
  inflating: /content/temp_dataset/19.wav  
  inflating: /conte

In [ ]:
loc = "%s/logs/%s" % (now_dir, experiment_name)
!zip -r rvcLogs.zip "{loc}"
DATASET_PATH_DRIVE = "/content/drive/MyDrive/rvcDisconnected/" + experiment_name
!mkdir -p "{DATASET_PATH_DRIVE}"
!cp /content/Mangio-RVC-Fork/rvcLogs.zip "{DATASET_PATH_DRIVE}"

updating: content/Mangio-RVC-Fork/logs/vivek2024/ (stored 0%)
updating: content/Mangio-RVC-Fork/logs/vivek2024/1_16k_wavs/ (stored 0%)
updating: content/Mangio-RVC-Fork/logs/vivek2024/1_16k_wavs/16_7.wav (deflated 6%)
updating: content/Mangio-RVC-Fork/logs/vivek2024/1_16k_wavs/10_3.wav (deflated 6%)
updating: content/Mangio-RVC-Fork/logs/vivek2024/1_16k_wavs/7_10.wav (deflated 6%)
updating: content/Mangio-RVC-Fork/logs/vivek2024/1_16k_wavs/7_0.wav (deflated 6%)
updating: content/Mangio-RVC-Fork/logs/vivek2024/1_16k_wavs/11_3.wav (deflated 6%)
updating: content/Mangio-RVC-Fork/logs/vivek2024/1_16k_wavs/22_6.wav (deflated 6%)
updating: content/Mangio-RVC-Fork/logs/vivek2024/1_16k_wavs/8_10.wav (deflated 7%)
updating: content/Mangio-RVC-Fork/logs/vivek2024/1_16k_wavs/1_0.wav (deflated 6%)
updating: content/Mangio-RVC-Fork/logs/vivek2024/1_16k_wavs/2_3.wav (deflated 5%)
updating: content/Mangio-RVC-Fork/logs/vivek2024/1_16k_wavs/15_2.wav (deflated 6%)
updating: content/Mangio-RVC-Fork/logs

In [ ]:
import os

BACK_UP_DATASET_PATH = "/content/drive/MyDrive/rvcDisconnected/" + experiment_name

#Prevent people from loading the ZIP over existing files
ok=True
if(os.path.exists("/content/Mangio-RVC-Fork/logs/"+experiment_name+"/2a_f0")):
  print("Dataset files already loaded, skipping.")
  ok=False

if ok:
  !unzip "{BACK_UP_DATASET_PATH}/rvcLogs.zip" -d /

Archive:  /content/drive/MyDrive/rvcDisconnected/viveknewer/rvcLogs.zip
   creating: /content/Mangio-RVC-Fork/logs/viveknewer/
  inflating: /content/Mangio-RVC-Fork/logs/viveknewer/preprocess.log  
   creating: /content/Mangio-RVC-Fork/logs/viveknewer/3_feature768/
  inflating: /content/Mangio-RVC-Fork/logs/viveknewer/3_feature768/19_3.npy  
  inflating: /content/Mangio-RVC-Fork/logs/viveknewer/3_feature768/6_6.npy  
  inflating: /content/Mangio-RVC-Fork/logs/viveknewer/3_feature768/20_1.npy  
  inflating: /content/Mangio-RVC-Fork/logs/viveknewer/3_feature768/20_6.npy  
  inflating: /content/Mangio-RVC-Fork/logs/viveknewer/3_feature768/20_11.npy  
  inflating: /content/Mangio-RVC-Fork/logs/viveknewer/3_feature768/21_0.npy  
  inflating: /content/Mangio-RVC-Fork/logs/viveknewer/3_feature768/25_0.npy  
  inflating: /content/Mangio-RVC-Fork/logs/viveknewer/3_feature768/4_3.npy  
  inflating: /content/Mangio-RVC-Fork/logs/viveknewer/3_feature768/9_4.npy  
  inflating: /content/Mangio-RVC-F

In [ ]:
import os
import sys
import traceback
import numpy as np
import faiss

exp_dir = "%s/logs/%s" % (now_dir, experiment_name)
os.makedirs(exp_dir, exist_ok=True)
feature_dir = (
    "%s/3_feature256" % (exp_dir)
    if model_architecture == "v1"
    else "%s/3_feature768" % (exp_dir)
)
print(feature_dir)
if not os.path.exists(feature_dir):
  raise Exception("No features exist for this model yet. Did you run Feature Extraction?")
listdir_res = list(os.listdir(feature_dir))
if len(listdir_res) == 0:
  raise Exception("No features exist for this model yet. Did you run Feature Extraction?")

try:
  from sklearn.cluster import MiniBatchKMeans
except:
  print("Due to a bug with Colab, we will need to reinstall Numpy real quick. Give me a sec!")
  !pip install -U numpy
  print("Numpy reinstalled. Please restart the runtime, and then re-run the \"Set Training Variables\" cell to continue.")
  sys.exit()
else:
  print("Proper Numpy version detected.")

infos=[]
npys=[]
for name in sorted(listdir_res):
  phone = np.load("%s/%s" % (feature_dir, name))
  npys.append(phone)
big_npy = np.concatenate(npys, 0)
big_npy_idx = np.arange(big_npy.shape[0])
np.random.shuffle(big_npy_idx)
if big_npy.shape[0] > 2e5 or force_mbkm:
  print("Trying doing kmeans %s shape to 10k centers." % big_npy.shape[0])
  try:
    big_npy = (
        MiniBatchKMeans(
            n_clusters=10000,
            verbose=True,
            batch_size=256,
            compute_labels = False,
            init="random"
        )
        .fit(big_npy)
        .cluster_centers_

    )
  except:
    info = traceback.format_exc()
    print(info)

np.save("%s/total_fea.npy" % exp_dir, big_npy)
n_ivf = min(int(16*np.sqrt(big_npy.shape[0])), big_npy.shape[0] // 39)
print("%s,%s" % (big_npy.shape, n_ivf))
index = faiss.index_factory(256 if model_architecture == "v1" else 768, "IVF%s,Flat" % n_ivf)
print("Training index...")
index_ivf = faiss.extract_index_ivf(index)
index_ivf.nprobe = 1
index.train(big_npy)
faiss.write_index(
    index,
    "%s/trained_IVF%s_Flat_nprobe_%s_%s_%s.index" % (exp_dir, n_ivf, index_ivf.nprobe, experiment_name, model_architecture)
)
print("Adding...")
batch_size_add = 8192
for i in range(0, big_npy.shape[0], batch_size_add):
  index.add(big_npy[i:i+batch_size_add])
faiss.write_index(
    index,
    "%s/added_IVF%s_Flat_nprobe_%s_%s_%s.index"
    % (exp_dir, n_ivf, index_ivf.nprobe, experiment_name, model_architecture)
)

npr = index_ivf.nprobe

print("Saving files to Drive...")
DATASET_PATH_DRIVE = "/content/drive/MyDrive/rvcDisconnected/" + experiment_name
if(not os.path.exists(DATASET_PATH_DRIVE)):
  !mkdir -p "{DATASET_PATH_DRIVE}"
DATASET_PATH_COLAB = "/content/Mangio-RVC-Fork/logs/" + experiment_name
if(save_extra_files_to_drive):
  !cp "{DATASET_PATH_COLAB}/total_fea.npy" "{DATASET_PATH_DRIVE}"
  !cp "{DATASET_PATH_COLAB}/trained_IVF{n_ivf}_Flat_nprobe_{npr}_{experiment_name}_{model_architecture}.index" "{DATASET_PATH_DRIVE}"
!cp "{DATASET_PATH_COLAB}/added_IVF{n_ivf}_Flat_nprobe_{npr}_{experiment_name}_{model_architecture}.index" "{DATASET_PATH_DRIVE}"

print("All done! Your index file has completed training.")
try:
  firsttry
except:
  print("If you had to restart the runtime, disconnect and delete the runtime in order to continue. (Restarting the runtime again will not work.)")



/content/Mangio-RVC-Fork/logs/vivek2024/3_feature768
Proper Numpy version detected.
(9472,),242
Training index...


ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
import os

DATASET_PATH_DRIVE = "/content/drive/MyDrive/rvcDisconnected/" + experiment_name
DATASET_PATH_COLAB = "/content/Mangio-RVC-Fork/logs/" + experiment_name
STEPCOUNT = 2333333

print("Copying model files...")
!cp "{DATASET_PATH_DRIVE}/D_{STEPCOUNT}.pth" "{DATASET_PATH_COLAB}"
!cp "{DATASET_PATH_DRIVE}/G_{STEPCOUNT}.pth" "{DATASET_PATH_COLAB}"
!cp "{DATASET_PATH_DRIVE}/config.json" "{DATASET_PATH_COLAB}"

print("Copying Tensorboard TFEVENT files...")
for r, _, f in os.walk(DATASET_PATH_DRIVE):
  for name in f:
    if(name.startswith("events.out.tfevents")):
      !cp "{DATASET_PATH_DRIVE}/{name}" "{DATASET_PATH_COLAB}"

print("All done. Welcome back!")

Copying model files...
Copying Tensorboard TFEVENT files...
All done. Welcome back!


In [ ]:
#TRAINING
import os
import math
from random import shuffle

assert 'model_architecture' in locals(),

assert 'pretrain_type' in locals(),

save_frequency = 10
total_epochs = 1850
batch_size = 8
save_only_latest_ckpt = True
cache_all_training_sets = False
save_small_final_model = True


assert save_frequency!=None,
assert save_frequency>0,
if(save_frequency>50):print(f"...A save frequency of {save_frequency}? A bit high, but... alright then.")
assert total_epochs!=None, "
assert total_epochs>0,
if(total_epochs>10000):print(f"...A total epoch count of of {total_epochs}? This is going to overtrain, but... alright then.")
assert batch_size!=None,
assert batch_size>0,
assert batch_size<=40,

pretrained_base = "pretrained/" if model_architecture == "v1" else "pretrained_v2/"
unpt = f"_{pretrain_type}" if pretrain_type!="original" else ""

pretrainedD = f"{pretrained_base}f0D{target_sample_rate}{unpt}.pth"
pretrainedG = f"{pretrained_base}f0G{target_sample_rate}{unpt}.pth"

log_interval = 1
liFolderPath = os.path.join(exp_dir, "1_16k_wavs")
if(os.path.exists(liFolderPath) and os.path.isdir(liFolderPath)):
  wav_files = [f for f in os.listdir(liFolderPath) if f.endswith(".wav")]
  if wav_files:
    sample_size = len(wav_files)
    log_interval = math.ceil(sample_size / batch_size)
    if log_interval > 1:
      log_interval += 1

if log_interval > 250 and not use_manual_stepToEpoch:
  print(f"That's a big dataset you got there. Log interval normalized to 200 steps from {log_interval} steps.")
  log_interval = 200

if use_manual_stepToEpoch:
  log_interval = manual_stepToEpoch

cmd = "python train_nsf_sim_cache_sid_load_pretrain.py -e \"%s\" -sr %s -f0 %s -bs %s -g %s -te %s -se %s %s %s -l %s -c %s -sw %s -v %s -li %s" % (
    experiment_name,
    target_sample_rate,
    1,
    batch_size,
    0,
    total_epochs,
    save_frequency,
    "-pg %s" % pretrainedG if pretrainedG != "" else "\b",
    "-pd %s" % pretrainedD if pretrainedD != "" else "\b",
    1 if save_only_latest_ckpt else 0,
    1 if cache_all_training_sets else 0,
    1 if save_small_final_model else 0,
    model_architecture,
    log_interval,
)
print(cmd)

gt_wavs_dir = f"{exp_dir}/0_gt_wavs"
feature_dir = (
  f"{exp_dir}/3_feature256"
  if model_architecture == "v1"
  else f"{exp_dir}/3_feature768"
)
f0_dir = f"{exp_dir}/2a_f0"
f0nsf_dir = f"{exp_dir}/2b-f0nsf"
names = (
  set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)])
  & set([name.split(".")[0] for name in os.listdir(feature_dir)])
  & set([name.split(".")[0] for name in os.listdir(f0_dir)])
  & set([name.split(".")[0] for name in os.listdir(f0nsf_dir)])
)
opt = []
for name in names:
  opt.append(
    "%s/%s.wav|%s/%s.npy|%s/%s.wav.npy|%s/%s.wav.npy|%s"
    % (
      gt_wavs_dir.replace("\\", "\\\\"),
      name,
      feature_dir.replace("\\", "\\\\"),
      name,
      f0_dir.replace("\\", "\\\\"),
      name,
      f0nsf_dir.replace("\\", "\\\\"),
      name,
      speaker_id,
    )
  )
fea_dim = 256 if model_architecture == "v1" else 768
for _ in range(2):
  opt.append(
      f"{now_dir}/logs/mute/0_gt_wavs/mute{target_sample_rate}.wav|{now_dir}/logs/mute/3_feature{fea_dim}/mute.npy|{now_dir}/logs/mute/2a_f0/mute.wav.npy|{now_dir}/logs/mute/2b-f0nsf/mute.wav.npy|{speaker_id}"
  )
shuffle(opt)
with open(f"{exp_dir}/filelist.txt", "w") as f:
  f.write("\n".join(opt))
print("Mute filelist written. Best of luck training!")

os.chdir('/content/Mangio-RVC-Fork')

log_file = f"{exp_dir}/training_log.txt"
print(f"Training logs will be saved to: {log_file}")

import subprocess
print("Starting training process...")
with open(log_file, "w") as f:
    process = subprocess.Popen(
        cmd,
        shell=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )

    for line in process.stdout:
        print(line, end='')
        f.write(line)
        f.flush()

    process.wait()

print(f"Training completed. Check log file at {log_file}")

python train_nsf_sim_cache_sid_load_pretrain.py -e "viveknewer" -sr 40k -f0 1 -bs 8 -g 0 -te 1850 -se 10 -pg pretrained_v2/f0G40k_TITAN.pth -pd pretrained_v2/f0D40k_TITAN.pth -l 1 -c 0 -sw 1 -v v2 -li 38
Mute filelist written. Best of luck training!
Training logs will be saved to: /content/Mangio-RVC-Fork/logs/viveknewer/training_log.txt
Starting training process...
INFO:viveknewer:{'train': {'log_interval': 38, 'seed': 1234, 'epochs': 20000, 'learning_rate': 0.0001, 'betas': [0.8, 0.99], 'eps': 1e-09, 'batch_size': 8, 'fp16_run': True, 'lr_decay': 0.999875, 'segment_size': 12800, 'init_lr_ratio': 1, 'warmup_epochs': 0, 'c_mel': 45, 'c_kl': 1.0}, 'data': {'max_wav_value': 32768.0, 'sampling_rate': 40000, 'filter_length': 2048, 'hop_length': 400, 'win_length': 2048, 'n_mel_channels': 125, 'mel_fmin': 0.0, 'mel_fmax': None, 'training_files': './logs/viveknewer/filelist.txt'}, 'model': {'inter_channels': 192, 'hidden_channels': 192, 'filter_channels': 768, 'n_heads': 2, 'n_layers': 6, 'ke

KeyboardInterrupt: 

In [ ]:
import os

DATASET_PATH_DRIVE = "/content/drive/MyDrive/rvcDisconnected/" + experiment_name
DATASET_PATH_COLAB = "/content/Mangio-RVC-Fork/logs/" + experiment_name
STEPCOUNT = 2333333

print("Copying model files...")
!cp "{DATASET_PATH_DRIVE}/D_{STEPCOUNT}.pth" "{DATASET_PATH_COLAB}"
!cp "{DATASET_PATH_DRIVE}/G_{STEPCOUNT}.pth" "{DATASET_PATH_COLAB}"
!cp "{DATASET_PATH_DRIVE}/config.json" "{DATASET_PATH_COLAB}"

print("Copying Tensorboard TFEVENT files...")
for r, _, f in os.walk(DATASET_PATH_DRIVE):
  for name in f:
    if(name.startswith("events.out.tfevents")):
      !cp "{DATASET_PATH_DRIVE}/{name}" "{DATASET_PATH_COLAB}"

print("All done. Welcome back!")

Copying model files...
Copying Tensorboard TFEVENT files...
All done. Welcome back!


In [ ]:
import os

DATASET_PATH_DRIVE = "/content/drive/MyDrive/rvcDisconnected/" + experiment_name
DATASET_PATH_COLAB = "/content/Mangio-RVC-Fork/logs/" + experiment_name
if(not os.path.exists(DATASET_PATH_DRIVE)):
  !mkdir -p "{DATASET_PATH_DRIVE}"

STEPCOUNT = 2333333
EPOCHCOUNT = 2333333

finished=False
potential="/content/Mangio-RVC-Fork/weights/"+experiment_name+".pth"
if os.path.exists(potential):
  finished = True

print("Detecting latest model...")
if(not manual_save):
  currentMax = 0
  for r, _, f in os.walk("/content/Mangio-RVC-Fork/weights/"):
    for name in f:
      if(name.endswith(".pth") and (name!=experiment_name+".pth")):
        if(name.find(experiment_name)==-1):
          continue
        pot = name.split('_')
        ep=pot[len(pot)-2][1:]
        if(not ep.isdecimal()):
          continue
        ep=int(ep)
        if ep>currentMax:
          currentMax=ep
          step=pot[len(pot)-1].split('.')
          step=int(step[0][1:])
          EPOCHCOUNT=ep
          STEPCOUNT=step

TSTEP = STEPCOUNT
if(not skip_models):
  print("Copying model files...")
  if(save_only_latest_ckpt):
    TSTEP=2333333
  !cp "{DATASET_PATH_COLAB}/D_{TSTEP}.pth" "{DATASET_PATH_DRIVE}"
  !cp "{DATASET_PATH_COLAB}/G_{TSTEP}.pth" "{DATASET_PATH_DRIVE}"
  !cp "{DATASET_PATH_COLAB}/config.json" "{DATASET_PATH_DRIVE}"

print("Copying Tensorboard TFEVENT files...")
for r, d, f in os.walk(DATASET_PATH_COLAB):
  for name in f:
    if(name.startswith("events.out.tfevents") and os.path.exists(os.path.join(DATASET_PATH_COLAB, name))):
      !cp "{DATASET_PATH_COLAB}/{name}" "{DATASET_PATH_DRIVE}"

print("Copying weight file...")
if(finished):
  !cp "{potential}" "{DATASET_PATH_DRIVE}"
else:
  !cp "/content/Mangio-RVC-Fork/weights/{experiment_name}_e{EPOCHCOUNT}_s{STEPCOUNT}.pth" "{DATASET_PATH_DRIVE}"

print("All done!")

Detecting latest model...
Copying model files...
Copying Tensorboard TFEVENT files...
Copying weight file...
All done!
